# Food Bank Problem

In [1]:
import sys
import numpy as np
import plotly.express as px
import pandas as pd

## OPT - Waterfilling

In [2]:
## Water-filling Algorithm for sorted demands
def waterfilling_sorted(d,b):
    n = np.size(d)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        equal_allocation = bundle_remaining/(n-i)
        if d[i]<equal_allocation:
            allocations[i] = bundle_remaining if i==n-1 else d[i]
        else:
            allocations[i] = equal_allocation
        bundle_remaining -= allocations[i]
    return allocations

In [3]:
## Water-filling Algorithm for general demands
def waterfilling(d,b):
    n = np.size(d)
    sorted_indices = np.argsort(d)
    sorted_demands = np.sort(d)
    sorted_allocations = waterfilling_sorted(sorted_demands, b)
    allocations = np.zeros(n)
    for i in range(n):
        allocations[sorted_indices[i]] = sorted_allocations[i]
    return allocations

In [4]:
## Tests
assert list(waterfilling(np.zeros(0), 5)) == []
assert list(waterfilling(np.array([1,2,3,4]), 10)) == [1,2,3,4]
assert list(waterfilling(np.array([3,4,1,2]), 10)) == [3,4,1,2]
assert list(waterfilling(np.array([1,2,3,4]), 8)) == [1,2,2.5,2.5]
assert list(waterfilling(np.array([3,1,4,2]), 8)) == [2.5,1,2.5,2]
assert list(waterfilling(np.array([3,6,5,6]), 8)) == [2,2,2,2]

## Online Algorithms

In [5]:
## Online Water-filling taking minimum of realized demand and predetermeined allocation
def waterfilling_online_1(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    prior_allocations_assignment = waterfilling(demands_predicted,b)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = min(prior_allocations_assignment[i], demands_realized[i]) if i!=n-1 else bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [6]:
## Tests
assert list(waterfilling_online_1(np.zeros(0), np.zeros(0), 5)) == []
assert list(waterfilling_online_1(np.array([1,2,3,4]), np.array([5,5,5,5]), 10)) == [1,2,3,4]
assert list(waterfilling_online_1(np.array([3,1,4,2]), np.array([2,3,2.5,1]), 8)) == [2,1,2.5,2.5]


In [83]:
def insert_sorted(lst, element):
    for i in range(np.size(lst)):
        if element<=lst[i]:
            np.append(np.append(lst[:i],element),lst[i:])
            return lst
    return np.append(lst,element)

In [96]:
def waterfilling_faster(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_indices = np.argsort(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    index_dict = {k: v for v, k in np.ndenumerate(sorted_indices)}
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = np.delete(sorted_demands, np.argwhere(sorted_demands==demands_predicted[i])[0][0])
        print(waterfilling_sorted(insert_sorted(sorted_demands,demands_realized[i]), bundle_remaining))
        allocations[i] = waterfilling_sorted(insert_sorted(sorted_demands,demands_realized[i]), bundle_remaining)[index_dict[i][0]]
        del index_dict[i]
        bundle_remaining -= allocations[i]
    print(allocations)
    return allocations
    

In [97]:
## Online Water-filling algorithm where each agent solves waterfilling with realized current demand and expected following demands
def waterfilling_online_2(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = waterfilling(np.append(demands_realized[i],demands_predicted[i+1:]), bundle_remaining)[0]
        bundle_remaining -= allocations[i]
    return allocations

In [98]:
## Tests 
assert list(waterfilling_online_2(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_2(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

assert list(waterfilling_faster(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_faster(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_faster(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_faster(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]


[]
[2. 3. 3. 3.]
[3. 3. 3.]
[3. 3.]


IndexError: index 2 is out of bounds for axis 0 with size 2

In [9]:
## Online Water-filling algorithm where each agent is assigned infinite demand while finding optimal solution and allocation is readjusted
def waterfilling_online_3(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        future_allocations = waterfilling(np.append(np.Inf,demands_predicted[i+1:]), bundle_remaining)
        if future_allocations[0]>demands_realized[i] and i!=n-1:
            allocations[i] = demands_realized[i]
        else:
            allocations[i] = future_allocations[0]
        bundle_remaining -= allocations[i]
    return allocations

In [10]:
## Tests 
assert list(waterfilling_online_3(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_3(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

## Objective Function

In [11]:
## Calculate log of Nash welfare for objective function
def objective(demands, allocation):
    welfare_sum = 0
    for i in range(np.size(demands)):
        welfare_sum += np.log(min(1,allocation[i]/demands[i]))
    return welfare_sum

## Experiment

In [52]:
def make_demands_uniform_distribution(num_towns, demand_range):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.uniform(0, demand_range[i])
        expected_demands[i] = demand_range[i]/2
    return demands, expected_demands


In [53]:
def make_demands_exponential_distribution(num_towns, demand_mean):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.exponential(demand_mean[i])
        expected_demands[i] = demand_range[i]
    return demands, expected_demands


In [54]:
num_iterations = 10
num_towns_range = 100
demands_max = 20
max_n = 1000


### Regret for Uniform Distribution of Demands

In [50]:
data_dict = {'NumTowns':[],'Alg_1':[],'Alg_2':[],'Alg_3':[]}

for n in range(max_n):
    for i in range(num_iterations):
        data_dict['NumTowns'].append(n)
        demand_range = np.random.uniform(0, demands_max, n)
        town_demands, town_expected_demands = make_demands_uniform_distribution(n, demand_range)
        budget = np.sum(town_expected_demands)

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
            if j==1:
                data_dict['Alg_2'].append(opt - objective(town_demands, waterfilling_online_1(town_expected_demands,town_demands,budget)))
            if j==2:
                data_dict['Alg_3'].append(opt - objective(town_demands, waterfilling_online_2(town_expected_demands,town_demands,budget)))
df = pd.DataFrame(data_dict).melt(id_vars="NumTowns")
fig = px.scatter(df, x="NumTowns", y="value", color='variable')
fig.show()

### Regret for Exponential Distribution of Demands

In [55]:
data_dict = {'NumTowns':[],'Alg_1':[],'Alg_2':[],'Alg_3':[]}

for n in range(max_n):
    for i in range(num_iterations):
        data_dict['NumTowns'].append(n)
        demand_mean = np.random.uniform(0, demands_max/2, n)
        town_demands, town_expected_demands = make_demands_exponential_distribution(n, demand_range)
        budget = np.sum(town_expected_demands)

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
            if j==1:
                data_dict['Alg_2'].append(opt - objective(town_demands, waterfilling_online_1(town_expected_demands,town_demands,budget)))
            if j==2:
                data_dict['Alg_3'].append(opt - objective(town_demands, waterfilling_online_2(town_expected_demands,town_demands,budget)))
df = pd.DataFrame(data_dict).melt(id_vars="NumTowns")
fig = px.scatter(df, x="NumTowns", y="value", color='variable')
fig.show()